In [166]:
#1 Reading in the csv-files
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
movieData= pd.read_csv('./Data/movies.csv', usecols=["movieId", 'title'], dtype={'movieId': 'int32', 'title': 'str'})
ratingData = pd.read_csv('./Data/ratings.csv', usecols=["userId", "movieId", "rating"], dtype ={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
ratingData.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [167]:
len(ratingData)

27753444

In [168]:
movies_with_few_ratings = ratingData["movieId"].value_counts().sort_values(ascending=True).head(25000)
ratingData = ratingData[~ratingData['movieId'].isin(movies_with_few_ratings)]
len(ratingData)

27623784

In [169]:
#Getting rid of some data, trying with only movies rated 5.
ratingData = ratingData.drop(ratingData[ratingData.rating < 4].index)
len(ratingData)

13775094

In [170]:
ratingData.index

Int64Index([       3,        4,        7,        8,       11,       12,
                  21,       23,       25,       26,
            ...
            27753434, 27753435, 27753436, 27753437, 27753438, 27753439,
            27753440, 27753441, 27753442, 27753443],
           dtype='int64', length=13775094)

In [171]:
movies_users=ratingData.pivot(index='movieId', columns='userId', values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

C:\Users\fredr\.virtualenvs\Maskininl-rning-Fredrik-Hegardt-mmrJtyaC\lib\site-packages\pandas\core\reshape\reshape.py:130: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


IndexError: index 1719333441 is out of bounds for axis 0 with size 1719332727

In [ ]:
model_knn=NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [ ]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [ ]:
def recommender(movie_name, data, model, n_recommendations):
    model.fit(data)
    idx=process.extractOne(movie_name, movieData['title'])[2]
    print("Movie Selected: ", movieData['title'][idx], "Index: ",idx)
    print("Searching for recommendations...")
    distances, indices = model.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    print(movieData.loc[indices[0][1:]])

recommender('Shawshank redemption', mat_movies_users, model_knn, 5)

Movie Selected: 

KeyboardInterrupt: 